## Uploading the data and confirming data types 

In [2]:
# right, let us first upload the data

# i have added this sample dataset in Github for you to try on

# we will be working throughout with the pandas library

import pandas as pd

df = pd.read_excel(r'C:\Users\Arun\Desktop\Medium Articles\Calculating premium earnings in insurance\data.xlsx',header=2)

#check if the data is uploaded correctly

df.head()

,Policy No,Inception date,Expiry date,Premium
0,ABC1,2018-01-01,2018-12-31,1242.936780
1,ABC2,2018-01-02,2019-01-01,1157.139103
2,ABC3,2018-01-03,2019-01-02,930.306625
3,ABC4,2018-01-04,2019-01-03,1087.214513
4,ABC5,2018-01-05,2019-01-04,1150.098881


In [3]:
# we are going to use pandas datetime libraries, so we need to make sure inception and expiry dates column data types are accurate

# let us check with info method

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 4 columns):
Policy No          730 non-null object
Inception date     730 non-null datetime64[ns]
Expiry date        730 non-null datetime64[ns]
Premium            730 non-null float64
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 22.9+ KB


Yes, both inception and expiry dates are datetime objects as required

## Automatically finding Month End Dates or Calculation dates where earnings need to be computed

In [4]:
# this is the key bit for calculating the earnings and for code reusability

# we are going to find out with our data, all the dates or time intervals for which earnings have to be derived

# First, we will extract inception date column from our dataframe 'df' and save it to df_triangledates dataframe

df_triangledates=df.loc[:,['Inception date ']]

# We will now include Month End Date column to each row

# import the pandas month end package which will help us to do it

from pandas.tseries.offsets import MonthEnd

df_triangledates['Month End Date']=pd.to_datetime(df_triangledates['Inception date '], format="%Y%m") + MonthEnd(0)

# now find the first month end date from which we need to do computations

# we can just find this using min on this column

df_firstmonth = df_triangledates['Month End Date'].min()

# also find today's date

today=pd.to_datetime('today')

print ("first month for earnings computation:", df_firstmonth)


first month for earnings computation: 2018-01-31 00:00:00


In [5]:
# So why did we find out the start month and today's date

# well, we are going to iterate over every month end date from beginning to end of previous month

# this is usually how we would compute monthly triangles

# make the first month end date, the first date of the iterator

use_date = df_firstmonth

# import the relative delta package to iterate over month end dates till last month

from dateutil.relativedelta import *

# define an empty list

dates =[use_date]

# start a while loop iterator

while use_date < today:

      use_date = use_date + relativedelta(months=+1)

      dates.append(use_date)
         
 
 # i like to use dataframes to analyse stuff, so let us convert the dates list to a dataframe

df_bdx_date3=pd.DataFrame(dates,columns=['Month dates'])

# so did this do the trick? , let us check

df_bdx_date3


,Month dates
0,2018-01-31
1,2018-02-28
2,2018-03-28
3,2018-04-28
4,2018-05-28
5,2018-06-28
6,2018-07-28
7,2018-08-28
8,2018-09-28
9,2018-10-28


In [6]:
# ah, we have a  problem now as the iterator simply seems to add up everything by 31 days, but we are interested in the month end dates

# we will use the month end package again to resolve this

df_bdx_date3['Month End Date']=pd.to_datetime(df_bdx_date3['Month dates'], format="%Y%m") + MonthEnd(0)

# let us drop the last date or end of current month

df_bdx_date5=df_bdx_date3[df_bdx_date3['Month End Date']<today]

#let us drop month dates column as we don't need this

df_bdx_date = df_bdx_date5.drop(columns=['Month dates'])

# see if everything has worked

df_bdx_date

,Month End Date
0,2018-01-31
1,2018-02-28
2,2018-03-31
3,2018-04-30
4,2018-05-31
5,2018-06-30
6,2018-07-31
7,2018-08-31
8,2018-09-30
9,2018-10-31


In [7]:
# well, we have found the dates that we need to calculate on, but how do we actually do our earnings calculations

# i am going to use the merge function to add these dates to every single row of our original dataframe

# because essentially this is what earnings calculations have to do, we need to figure out the earnings for every record or insurance policy at every month end date

# to merge, we add a dummy id of 1 for merging to the original dataframe df

df_bdx_date['Date id']=1

# Repeat the same in the df dataframe

df['Date id']=1

#Merge these two dataframes so that all policies have all month end dates to every row as duplicates.

df_triangle3=pd.merge(df,df_bdx_date,left_on=['Date id'], right_on=['Date id'])

# Apply a boolean logic and create a new column to reduce row count. 

# There is no requirement to calculate earned premium for a policy which incepts after a month end date

#Then filter out the True statements

df_triangle3['Inception Date greater']=df_triangle3['Inception date ']<=df_triangle3['Month End Date']

df_triangle5=df_triangle3[df_triangle3['Inception Date greater']==1]

# Drop out this boolean column afterwards, we do not need it anymore

df_triangle7=df_triangle5.drop(columns=['Inception Date greater','Date id'])

# let us how this all looks like now

df_triangle7.head()


,Policy No,Inception date,Expiry date,Premium,Month End Date
0,ABC1,2018-01-01,2018-12-31,1242.93678,2018-01-31
1,ABC1,2018-01-01,2018-12-31,1242.93678,2018-02-28
2,ABC1,2018-01-01,2018-12-31,1242.93678,2018-03-31
3,ABC1,2018-01-01,2018-12-31,1242.93678,2018-04-30
4,ABC1,2018-01-01,2018-12-31,1242.93678,2018-05-31


# Now let us calculate the earnings by 365th method

So this is where all our earlier work to bring in month end dates across each policy will bear fruit, and we will use the df_triangle7 dataframe to calculate earnings

Here I will be using the 365th method to calculate earnings , our calculations are derived as follows:

Earned Exposure = (Month End date - Inception Date) / (Expiry Date - Inception Date)

limit earned exposure to 1 (Maximum)

the denominator formula will also account for multi year insurance policies

So finally Earned Premium will be

Earned Premium (EP) = Premium * Earned Exposure



In [8]:
# i am going to break the numerator and denominator steps here to keep things simple

# diff between dates creates a time delta series in pandas

# Use dt.days to access the no of days as an int data type

df_triangle7['Earned Exposure_num']=((df_triangle7['Month End Date']-df_triangle7['Inception date ']).dt.days)

# let us calculate the denominator value

df_triangle7['Earned Exposure_den']=(df_triangle7['Expiry date']-df_triangle7['Inception date ']).dt.days

# now divide these to find the earned exposure_without cap

df_triangle7['Earned Exposure-no cap']=df_triangle7['Earned Exposure_num']/df_triangle7['Earned Exposure_den']

# now this measure will be greater than one is some instances, so we need to limit it to one as earned exposure cannot be greater than one

df_triangle7['Earned Exposure'] = df_triangle7['Earned Exposure-no cap'].apply(lambda x: 1 if x > 1 else x)

# let us limit to columns we need

df_triangle_premium = df_triangle7.drop(columns=['Earned Exposure_num','Earned Exposure_den','Earned Exposure-no cap'])

# let us examine the data

df_triangle_premium.head()


,Policy No,Inception date,Expiry date,Premium,Month End Date,Earned Exposure
0,ABC1,2018-01-01,2018-12-31,1242.93678,2018-01-31,0.082418
1,ABC1,2018-01-01,2018-12-31,1242.93678,2018-02-28,0.159341
2,ABC1,2018-01-01,2018-12-31,1242.93678,2018-03-31,0.244505
3,ABC1,2018-01-01,2018-12-31,1242.93678,2018-04-30,0.326923
4,ABC1,2018-01-01,2018-12-31,1242.93678,2018-05-31,0.412088


In [9]:
# and here we go and find the earned premium

df_triangle_premium['Earned Premium'] = df_triangle_premium['Earned Exposure'] * df_triangle_premium['Premium']

# see, if this has worked and calcualted earnings for each policy at various month end dates

df_triangle_premium.head()

,Policy No,Inception date,Expiry date,Premium,Month End Date,Earned Exposure,Earned Premium
0,ABC1,2018-01-01,2018-12-31,1242.93678,2018-01-31,0.082418,102.439845
1,ABC1,2018-01-01,2018-12-31,1242.93678,2018-02-28,0.159341,198.050366
2,ABC1,2018-01-01,2018-12-31,1242.93678,2018-03-31,0.244505,303.904872
3,ABC1,2018-01-01,2018-12-31,1242.93678,2018-04-30,0.326923,406.344717
4,ABC1,2018-01-01,2018-12-31,1242.93678,2018-05-31,0.412088,512.199223


# Summarizing results and building the earnings triangle

We have computed the earnings for each and every policy. But in the real world, insurers would simply want to see the aggregated view across various time intervals. Let us make those aggregated view below

In [10]:
# we are going to aggregate policies by each inception month

# to do this, we are going to build in an inception month column with pandas month end

# add a column for inception month

df_triangle_premium['Inception Month'] = pd.to_datetime(df_triangle_premium['Inception date '], format="%Y%m") + MonthEnd(0)

# check if this has worked

df_triangle_premium.head()

,Policy No,Inception date,Expiry date,Premium,Month End Date,Earned Exposure,Earned Premium,Inception Month
0,ABC1,2018-01-01,2018-12-31,1242.93678,2018-01-31,0.082418,102.439845,2018-01-31
1,ABC1,2018-01-01,2018-12-31,1242.93678,2018-02-28,0.159341,198.050366,2018-01-31
2,ABC1,2018-01-01,2018-12-31,1242.93678,2018-03-31,0.244505,303.904872,2018-01-31
3,ABC1,2018-01-01,2018-12-31,1242.93678,2018-04-30,0.326923,406.344717,2018-01-31
4,ABC1,2018-01-01,2018-12-31,1242.93678,2018-05-31,0.412088,512.199223,2018-01-31


In [17]:
# Now df_triangles_premium represents the master earning file containing earnings for all policies for every possible calculation or month end date

# we will now group this dataframe again to arrive at our outputs

# first we can drop earned exposure column as we not longer need it

df_triangles_premium11=df_triangle_premium.drop(columns=['Earned Exposure'])

# now shrink the dataframe by grouping by month end or calculation date and inception month

df_triangles_groupby= df_triangles_premium11.groupby(by=['Month End Date','Inception Month'],as_index=False).sum()

# see, how this looks like

df_triangles_groupby.head()


,Month End Date,Inception Month,Premium,Earned Premium
0,2018-01-31,2018-01-31,30432.461130,1306.099854
1,2018-02-28,2018-01-31,30432.461130,3647.058403
2,2018-02-28,2018-02-28,28137.356889,1026.455944
3,2018-03-31,2018-01-31,30432.461130,6238.833939
4,2018-03-31,2018-02-28,28137.356889,3422.769305


In [18]:
# now to get the earnings triangle that we use are familiar with, we will use the pandas pivot function as below

# there will be  a few null values when we pivot, let us replaces this with a blank

earningspivot = df_triangles_groupby.pivot(index='Inception Month', columns='Month End Date', values='Earned Premium').fillna(" ")

# and here, we have below the earned premium triangle!

earningspivot

Month End Date,2018-01-31 00:00:00,2018-02-28 00:00:00,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,...,2019-06-30 00:00:00,2019-07-31 00:00:00,2019-08-31 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-30 00:00:00,2019-12-31 00:00:00,2020-01-31 00:00:00,2020-02-29 00:00:00,2020-03-31 00:00:00
Inception Month,,,,,,,,,,,,,,,,,,,,,
2018-01-31,1306.1,3647.06,6238.83,8747,11338.8,13846.9,16438.7,19030.5,21538.7,24130.4,...,30432.5,30432.5,30432.5,30432.5,30432.5,30432.5,30432.461130,30432.461130,30432.461130,30432.461130
2018-02-28,,1026.46,3422.77,5741.78,8138.1,10457.1,12853.4,15249.7,17568.7,19965.1,...,28137.4,28137.4,28137.4,28137.4,28137.4,28137.4,28137.356889,28137.356889,28137.356889,28137.356889
2018-03-31,,,1274.52,3897.9,6608.72,9232.09,11942.9,14653.7,17277.1,19987.9,...,31830.3,31830.3,31830.3,31830.3,31830.3,31830.3,31830.285478,31830.285478,31830.285478,31830.285478
2018-04-30,,,,1152.95,3620.6,6008.66,8476.32,10944,13332,15799.7,...,28975.1,28975.1,28975.1,28975.1,28975.1,28975.1,28975.069245,28975.069245,28975.069245,28975.069245
2018-05-31,,,,,1234.55,3766.38,6382.6,8998.83,11530.7,14146.9,...,30719.5,30719.5,30719.5,30719.5,30719.5,30719.5,30719.543916,30719.543916,30719.543916,30719.543916
2018-06-30,,,,,,1049.55,3464.66,5879.76,8216.96,10632.1,...,28358,28358,28358,28358,28358,28358,28358.008957,28358.008957,28358.008957,28358.008957
2018-07-31,,,,,,,1274.08,3812.26,6268.56,8806.74,...,28620.9,29803.1,29803.1,29803.1,29803.1,29803.1,29803.132784,29803.132784,29803.132784,29803.132784
2018-08-31,,,,,,,,1226.04,3638.55,6131.49,...,25592.5,28085.4,29271.9,29271.9,29271.9,29271.9,29271.868962,29271.868962,29271.868962,29271.868962
2018-09-30,,,,,,,,,1137.06,3562,...,22492.2,24917.1,27342,28473.5,28473.5,28473.5,28473.458552,28473.458552,28473.458552,28473.458552
